# Installation
From [the docs](http://qcodes.github.io/Qcodes/start/index.html):

1. Install [Anaconda](https://www.anaconda.com/download/). Get the Python 3 version. No need to install the VSCode stuff.
1. Save [environment.yml](https://raw.githubusercontent.com/QCoDeS/Qcodes/master/environment.yml) to the machine.
1. Run the following in an Anaconda Prompt:
```
conda env create -f environment.yml
activate qcodes
pip install qcodes
```
1. Close the Anaconda Prompt. You should now be able to open `Jupyter Notebook (qcodes)`.

In [ ]:
import qcodes as qc

# Instrument setup

At this point, each measurement setup needs to define which instruments they use. See [the docs](http://qcodes.github.io/Qcodes/user/intro.html#instrument). For demonstration purposes, here I just use dummy instruments `dac` and `dmm`.

In [ ]:
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument(name="dac", gates=['ch1', 'ch2'])
dmm = DummyInstrument(name="dmm", gates=['idc', 'ig'])

import math
import random
def semiconductor():
    gates = dac.ch1.get() + 0.6*dac.ch2.get()
    return 12.2*math.fabs(gates)*(1-math.exp(-(gates*gates)/3.2))*random.gauss(1, 0.001) + random.gauss(0, 0.001)
dmm.idc.get = semiconductor
dmm.ig.get = lambda: random.gauss(0, 0.01)

# Measurement

## Measure a single value

From this point onward, `station.snapshot()` will give the current state of every instrument. You can also call `snapshot()` on individual instruments, as well as parameters. You can use [`qc.Measure()`](http://qcodes.github.io/Qcodes/api/generated/qcodes.measure.Measure.html#qcodes.measure.Measure) to get the current state in a `DataSet`. The results will be saved in the usual place.

In [ ]:
_ = qc.Measure(dmm.idc, dmm.ig).run(name='measurement')

## 1D sweep

A basic sweep looks something like the following:

1. Create a `qc.SweepValues` or `qc.SweepFixedValues` either manually or by calling `sweep(start, stop, step)` on a parameter.
1. Create a `qc.Loop`, set a delay time between steps, and set which values to read each step.
1. Describe what to plot.
1. Run the loop, using a background task to update the plot.

Catch `KeyboardInterrupt` from the `loop.run()` call to make it not spit out a stacktrace when you abort the measurement. It will still save your data!

In [ ]:
from tp import TimeParameter

In [ ]:
# Sweep dac.ch1 from 0 to 5, 5 to -5, and then -5 to 0.
sv = dac.ch1.sweep(0, 5, 0.1)
sv.extend(dac.ch1.sweep(5, -5, 0.1))
sv.extend(dac.ch1.sweep(-5, 0, 0.1))

# Pause for 0.01 seconds between steps.
loop = qc.Loop(sv, delay=0.01).each(TimeParameter(), dmm.idc, dmm.ig)
data = loop.get_data_set(name='sweepvg')

# Subplot indices start from 1... yuck!
plot = qc.QtPlot()
plot.add(data.dmm_idc, subplot=1)
plot.add(data.dmm_ig, subplot=2)
plot.add(x=data.time, y=data.dmm_idc, subplot=3)

try:
    _ = loop.with_bg_task(plot.update, plot.save).run()
except KeyboardInterrupt:
    pass

## Continuously measure

The following "sweep" just watches for a given time. Useful for cooling your sample and so on. Follow [this issue](https://github.com/QCoDeS/Qcodes/issues/131) for more ideas. Hit the interrupt button in jupyter to cancel.

In [ ]:
from tp import loop_continuous

In [ ]:
loop = loop_continuous(3600, 0.1).each(dmm.idc, dmm.ig)
data = loop.get_data_set(name='cooldown')
plot = qc.QtPlot()
plot.add(data.dmm_idc, subplot=1)
plot.add(data.dmm_ig, subplot=2)

try:
    _ = loop.with_bg_task(plot.update, plot.save).run()
except KeyboardInterrupt:
    pass

# Data

Data is saved automatically in a file at `DataSet.location`. It is straightforward to configure the location formatter. Plots are also automatically saved. See [the docs](http://qcodes.github.io/Qcodes/dataset/index.html) for more info. There's also an interesting note on [interdependent parameters](http://qcodes.github.io/Qcodes/dataset/interdependentparams.html#).